## Проверка гипотез по регионам и федеральным округам

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.special import inv_boxcox

In [2]:
# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('final_dataset_2.csv')

### Равенство математических ожиданий продолжительности жизни по ФО

Для начала поработаем с показателем, который мы планируем предсказывать: рассмотрим, сильно ли отличается продолжительность жизни в зависимости от Федерального округа. Заодно изучим,полезен ли добавленный нами признак. 
Проверим гипотезу о ттом, что продолжительность жизни по регионам РФ не различается. Для этого проведем попарное сравнение нескольких ФО.   
   - Нулевая гипотеза ($H_0$): Средние значения продолжительности жизни в двух выбранных ФО равны.  
   - Альтернативная гипотеза ($H_1$): Средние значения продолжительности жизни в регионах из двух выбранных ФО различаются.  

In [4]:
siberian = data[data['ФО'] == 'siberian'].groupby('Год')['Средняя продолжительность жизни'].mean()
central = data[data['ФО'] == 'central'].groupby('Год')['Средняя продолжительность жизни'].mean()

In [5]:
t_statistic, p_value = stats.ttest_ind(central, siberian, equal_var=False)

print('t-статистика:', t_statistic)
print('p-value:', p_value)

print("Центральный и Сибирский ФО")
if p_value < 0.05:
    print("Есть статистически значимое различие")
else:
    print("Cтатистически значимое различие не выявлено")

t-статистика: 2.6106920506585163
p-value: 0.015409012873612545
Центральный и Сибирский ФО
Есть статистически значимое различие


In [6]:
volga = data[data['ФО'] == 'volga'].groupby('Год')['Средняя продолжительность жизни'].mean()
ural = data[data['ФО'] == 'ural'].groupby('Год')['Средняя продолжительность жизни'].mean()

In [7]:
t_statistic, p_value = stats.ttest_ind(siberian, volga, equal_var=False)

print('t-статистика:', t_statistic)
print('p-value:', p_value)

print("Сибирский и Приволжский ФО")
if p_value < 0.05:
    print("Есть статистически значимое различие")
else:
    print("Cтатистически значимое различие не выявлено")

t-статистика: -2.734817837960159
p-value: 0.011700031741607931
Сибирский и Приволжский ФО
Есть статистически значимое различие


In [8]:
t_statistic, p_value = stats.ttest_ind(central, ural, equal_var=False)

print('t-статистика:', t_statistic)
print('p-value:', p_value)

print("Центральный и Уральский ФО")
if p_value < 0.05:
    print("Есть статистически значимое различие")
else:
    print("Cтатистичски значимое различие не выявлено")

t-статистика: -0.06724212404204559
p-value: 0.9469713536276043
Центральный и Уральский ФО
Cтатистичски значимое различие не выявлено


In [9]:
t_statistic, p_value = stats.ttest_ind(siberian, ural, equal_var=False)

print('t-статистика:', t_statistic)
print('p-value:', p_value)

print("Сибирский и Уральский ФО")
if p_value < 0.05:
    print("Есть статистически значимое различие")
else:
    print("Cтатистически значимое различие не выявлено")

t-статистика: -2.906784231525054
p-value: 0.00821538100286463
Сибирский и Уральский ФО
Есть статистически значимое различие


Итак, мы сравнили пары ФО и видим, что в 3 из 4 пар есть статистически значимые различия, это говорит о том, что наш признак на самом деле может помочь в предсказании модели. С логической точки зрения это тоже вполне объяснимо, разные федеральные округа имеют разные социокультурные, экономические и экологические условия, что в любом случае откладывает отпечаток на уровне жизни населения и, как следствие, влияет на продолжительнсть жизни. Итак, мы сравнили пары ФО и видим, что в 3 из 4 пар есть статистически значимые различия, это говорит о том, что наш признак на самом деле может помочь в предсказании модели. С логической точки зрения это тоже вполне объяснимо, разные федеральные округа имеют разные социокультурные, экономические и экологические условия, что в любом случае откладывает отпечаток на уровне жизни населения и, как следствие, влияет на продолжительнсть жизни. 

Теперь сделаем проверку по всем регионам. После небольших приготовлений (отбор данных) запустим цикл, который позволит проверить в какой доле пар регионов нулевая гипотеза о равенстве средней продолжительности жизни соблюдается. 

In [67]:
fed_districts = ['central', 'southern', 'north_western', 'far_eastern', 'siberian', 'ural', 'volga', 'north_caucasian']


In [73]:
fed_districts_le = []
for fd in fed_districts:
    fd_le = data[data['ФО'] == fd].groupby('Год')['Средняя продолжительность жизни'].mean()
    fed_districts_le.append(fd_le)

In [77]:
h0 = 0
n_all = 0

for i in range (len(fed_districts_le)):
    fd_le_1 = fed_districts_le[i]
    for fd_le_2 in fed_districts_le[i+1:]:
        t_statistic, p_value = stats.ttest_ind(fd_le_1, fd_le_2, equal_var=False)
        if p_value > 0.5:
            h0 +=1
        n_all +=1

In [78]:
h0 / n_all

0.10714285714285714

Мы видим, что только в менее, чем 11% пар средние продолжительности жизни совпадают, что достаточно мало. 

Таким образом, наш предыдущий вывод оказался вполне обоснованным, средняя продолжительность жизни сильно различется в зависимости от Федерального окурга, а значит, что признак значимый, однако пока нет уверенности, какую роль он сыграет в нашей модели. 

### Равенство нулю математического ожидания прироста числа больничных коек 

Во время эксплорационного аналлиза данных мы увидели, что очень сильную крреляцию с предсказываемым признаком имеет показатель числа больничных коек на 10000 человек населения. Поэтому мы решили проверить, что прирст числа больничных коек распределен с нулевым мат ожиданием. Проверку гипотезы будем осуществлять в каждом регионе и потом посмотрим. на долю регинов, где мы нулевую гипотезу не отвергли.     
- Нулевая гипотеза $H_0$: $\mu_{\text{прирост числа больничных коек в регионе}} = 0$
- Альтернативная гипотеза $H_1$: $\mu_{\text{прирост числа больничных коек в регионе}} \neq 0$

In [10]:
def t_test_koyki(df):
    df = df.sort_values('Год')
    series = df['Число больничных коек'].diff()
    _, p_value = stats.ttest_1samp(series, 0, nan_policy= 'omit')
    return (p_value > 0.05) 

In [11]:
test_koyki = data.groupby('Регион')['Год', 'Число больничных коек'].apply(t_test_koyki)
test_koyki

Регион
Алтайский край                                  False
Амурская область                                 True
Архангельская область                            True
Архангельская область без автономного округа     True
Астраханская область                             True
                                                ...  
Ямало-Ненецкий автономный округ                 False
Ярославская область                              True
г. Москва                                        True
г. Санкт-Петербург                               True
г. Севастополь                                   True
Length: 87, dtype: bool

In [12]:
test_koyki.sum() / len(test_koyki)

0.8505747126436781

Итак, мы видим, что в 85% регионов наша гипотеза подтвердилась. То есть прирост коек действительно в основном клеблется около какого-то определенного значения, значит, что у нас есть +- устойчивый тренд.  
Далее проверим, есть ли диффузия, то есть узнаем, монотонный ли тренд. Опять же применим для этоого схожую функцию для всех регионов:

In [13]:
def t_test_koyki2(df):
    df = df.sort_values('Год')
    series = df['Число больничных коек'].diff().diff()
    _, p_value = stats.ttest_1samp(series, 0, nan_policy= 'omit')
    return (p_value > 0.05) 

In [14]:
test_koyki2 = data.groupby('Регион')['Год', 'Число больничных коек'].apply(t_test_koyki2)
test_koyki2.sum() / len(test_koyki2)

1.0

Мы видим, что диффузии нет в 100% регионов, то есть можно сказать, что значения в ряде не изменяются со временем или изменяются очень мало. В нашем случаае это хороший результат, так как если ряд не демонстрирует диффузии, это может указывать на наличие структуры, тренда или сезонных колебаний, которые поддерживают постоянные или практически постоянные значения ряда. В таком случае, ряд может быть более предсказуемым и иметь устойчивую модель, что нам и необходимо.

### Равенство средних зарплат в рамках одного ФО

Итак, сейчас мы раасмотрим гипотезу о том, что средние зарплаты в субъектах в одном ФО одинаковые. Мы предполагаем, что в пределах одного федерального округа можно ожидать схожих социоэкономических условий, что может привести к схожим уровням зарплат.  
Формулировка гипотезы:
   - Нулевая гипотеза ($H_0$): Средние зарплаты в рамках одного федерального округа равны. 
   - Альтернативная гипотеза ($H_1$): Средние зарплаты в рамках одного федерального округа различаются.
   
Для проверки этой гипотезы проведем попарные t-тесты для всех регионов в рамках одного округа. Проверим несколько федеральных округов и выведем для них результаты.

In [19]:
siberian_zp = data[data['ФО'] == 'siberian'][['Регион', 'Год', 'Среднедушевые денежные доходы населения']]

In [55]:
siberian_regions = sorted(list(set(siberian_zp['Регион'])))

results = pd.DataFrame(index=siberian_regions, columns=siberian_regions).fillna(0)

In [59]:
h0_ok = 0
n_h = 0

for i in range (len(siberian_regions)):
    region_1 = siberian_regions[i]
    zp_1 = siberian_zp[siberian_zp['Регион'] == region_1]['Среднедушевые денежные доходы населения']
    for region_2 in siberian_regions[i+1:]:
        zp_2 = siberian_zp[siberian_zp['Регион'] == region_2]['Среднедушевые денежные доходы населения']
        t_statistic, p_value = stats.ttest_ind(zp_1, zp_2)
        results.loc[region_1, region_2] = p_value
        results.loc[region_2, region_1] = p_value
        if p_value > 0.05:
            h0_ok += 1
        n_h +=1

In [61]:
h0_ok / n_h

0.7272727272727273

In [60]:
results

,Алтайский край,Забайкальский край,Иркутская область,Кемеровская область,Красноярский край,Омская область,Республика Алтай,Республика Бурятия,Республика Тыва,Республика Хакасия,Томская область
Алтайский край,0.000000,0.374693,0.292659,0.357308,0.012042,0.126271,0.498402,0.325063,0.055035,0.965995,0.068158
Забайкальский край,0.374693,0.000000,0.870300,0.964136,0.069514,0.502246,0.100721,0.928264,0.004858,0.321540,0.306905
Иркутская область,0.292659,0.870300,0.000000,0.823258,0.090701,0.606884,0.068308,0.940745,0.002783,0.242795,0.380720
Кемеровская область,0.357308,0.964136,0.823258,0.000000,0.048600,0.438674,0.076682,0.885712,0.002095,0.296656,0.252518
Красноярский край,0.012042,0.069514,0.090701,0.048600,0.000000,0.211749,0.001429,0.079039,0.000053,0.007612,0.385762
Омская область,0.126271,0.502246,0.606884,0.438674,0.211749,0.000000,0.020890,0.555799,0.000653,0.094537,0.699721
Республика Алтай,0.498402,0.100721,0.068308,0.076682,0.001429,0.020890,0.000000,0.079403,0.140095,0.490369,0.010303
Республика Бурятия,0.325063,0.928264,0.940745,0.885712,0.079039,0.555799,0.079403,0.000000,0.003347,0.273083,0.343233
Республика Тыва,0.055035,0.004858,0.002783,0.002095,0.000053,0.000653,0.140095,0.003347,0.000000,0.040126,0.000353
Республика Хакасия,0.965995,0.321540,0.242795,0.296656,0.007612,0.094537,0.490369,0.273083,0.040126,0.000000,0.048713


Можно подробнее изучить p-values для всех регионов (по главной диагонали стоят просто нули, мы не сравнивали регион сам с собой). В итоге видиим, что в 73% случаев наша гипотеза подтверждается и в рамках одного ФО действительно не различий по среднедушевому доходу граждан. Это означает, что нет статистически значимых различий в уровнях заработной платы между регионами внутри одного федерального округа. Сооветственно, мы можем делать вывод о схожих социоэкономических условиях внутри одного ФО и отсутствии систематических различий в заработной плате между его регионами.